<a href="https://colab.research.google.com/github/debbysonino/LamasDataHack/blob/master/DataLearn_2019_Scaffold_basefiledavid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

&nbsp; ![alt text](https://s3.amazonaws.com/monday.com/static/svg/monday-logos/monday-footer-logo.svg)

#Model scaffold
This notebook is intended to get you up and running faster.

It has the basic scaffold of an ML model, including:
* Data loading
* Feature extraction
* Columns transformation
* Training
* Evaluating
* Submitting results

###Getting our depnedncies (and data!)
First we'll import our relevant libraries

In [0]:
# General DS libraries we are going to need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# Importing our base model
# [REDACTED ML MODEL USED]

# Imports for working with our large dataset
from sklearn.utils.random import sample_without_replacement
from sklearn.model_selection import train_test_split

# We need those for data manipulation and getting our features ready for the model
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer

# These can be used to measure our model's performance
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# Ignore DataFrame assignment warnings
pd.options.mode.chained_assignment = None


We set a few constants to use later on for sampeling and running the model

In [0]:
#@title Model parameters { run: "auto" }
# n_neighbors = 7 #@param {type:"slider", min:1, max:30, step:1}
group_name = "Lamassim" #@param {type:"string"}
samples_num = 220000 #@param {type:"slider", min:0, max:1500000, step:10000}
n_jobs = -1 #@param {type:"slider", min:-1, max:32, step:1}
path_prefix = "https://storage.googleapis.com/mondaycom-datahack/final_sets" #@param ["https://storage.googleapis.com/mondaycom-datahack/final_sets", "https://mondaycom-datahack.s3.amazonaws.com/final_sets"] {allow-input: true}

Next we'll load all the different parts of our dataset

<br/>

_Our use my data loading [snippet](https://colab.research.google.com/drive/1_Y-sZ5eHIDlDUMuLCwfnbuJdLh0DTXmO#scrollTo=5HGlaJTEAYJu&line=23&uniqifier=1)!_

In [5]:
import os
import pandas as pd

# We define the datasets we want to load
datasets = ('accounts', 'users', 'events', 'subscriptions')
source_prefix = 'https://storage.googleapis.com/mondaycom-datahack/final_sets/'

local_dir = './datasets/datahack/'
file_prefix = 'train_'
file_suffix = ''
file_extension = 'csv'

# We create a directory for the datasets if it doesn't exist
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

# For each dataset we want, we check if we already downloaded it and fix it if we didn't
for dataset in datasets:
  if not os.path.isfile('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension)):
    !curl {source_prefix}{file_prefix}{dataset}{file_suffix}.{file_extension} --output {local_dir}{file_prefix}{dataset}{file_suffix}.{file_extension}

  # Load the datasets into a DataFrame using pandas
  globals()['{}{}'.format(file_prefix, dataset)] = pd.read_csv('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension), low_memory=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  204M  100  204M    0     0   142M      0  0:00:01  0:00:01 --:--:--  142M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  330M  100  330M    0     0   211M      0  0:00:01  0:00:01 --:--:--  211M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1215M  100 1215M    0     0   120M      0  0:00:10  0:00:10 --:--:--  148M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.1M  100 17.1M    0     0  35.2M      0 --:--:-- --:--:-- --:--:-- 35.1M


In [6]:
import os
import pandas as pd

# We define the datasets we want to load
datasets = ('accounts', 'users', 'events', 'subscriptions')
source_prefix = 'https://storage.googleapis.com/mondaycom-datahack/final_sets/'

local_dir = './datasets/datahack/'
file_prefix = 'test_'
file_suffix = ''
file_extension = 'csv'

# We create a directory for the datasets if it doesn't exist
if not os.path.exists(local_dir):
    os.makedirs(local_dir)

# For each dataset we want, we check if we already downloaded it and fix it if we didn't
for dataset in datasets:
  if not os.path.isfile('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension)):
    !curl {source_prefix}{file_prefix}{dataset}{file_suffix}.{file_extension} --output {local_dir}{file_prefix}{dataset}{file_suffix}.{file_extension}

  # Load the datasets into a DataFrame using pandas
  globals()['{}{}'.format(file_prefix, dataset)] = pd.read_csv('{}{}{}{}.{}'.format(local_dir, file_prefix, dataset, file_suffix, file_extension), low_memory=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.6M  100 10.6M    0     0  33.8M      0 --:--:-- --:--:-- --:--:-- 33.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.1M  100 17.1M    0     0  31.3M      0 --:--:-- --:--:-- --:--:-- 31.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.2M  100 63.2M    0     0  82.4M      0 --:--:-- --:--:-- --:--:-- 82.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  959k  100  959k    0     0  8802k      0 --:--:-- --:--:-- --:--:-- 8802k


We need to add our test sets to our train sets and work on both at the same time.

We'll split them back up before training and inference.

In [7]:
accounts = train_accounts.append(test_accounts)
users = train_users.append(test_users)
events = train_events.append(test_events)
subscriptions = train_subscriptions.append(test_subscriptions)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


###Feature engineering
In this block we add a new feature of `[REDACTED]` extracted from the user `[REDACTED]`

We also seperate all the `[REDACTED]` users into a different DataFrame

In [0]:
users['[REDACTED]'] = users['REDACTED'].apply(lambda x: "[REDACTED]")
[REDACTED] = users[users["[REDACTED]"] == "[REDACTED]"]

KeyError: ignored

Let's enrich our data a bit

In [29]:
# Joining the accounts with the [REDACTED] users
all_features = accounts.merge([REDACTED], on='account_id', suffixes=('_account', '_user'))

all_features = all_features.reset_index().set_index(['account_id', '[REDACTED]']).drop(columns='index')

# Adding the [REDACTED] in a seperate column
all_features['[REDACTED]'] = "[REDACTED]"

NameError: ignored

In [30]:
all_features==accounts

,account_id,account_name,browser,churn_date,churn_reason,collection_21_days,company_size,country,created_at,device,has_logo,industry,lead_score,max_team_size,min_team_size,mrr,os,paying,payment_currency,plan_id,region,subscription_started_at,team_size,time_diff,trial_start,user_description,user_goal,utm_cluster_id
0,True,True,True,False,False,True,False,True,True,True,True,False,True,True,True,False,True,True,True,False,True,False,False,True,True,False,False,True
1,True,True,False,False,False,True,False,True,True,True,True,False,True,True,True,False,True,True,True,False,True,False,False,False,True,False,False,False
2,True,True,True,False,False,True,False,True,True,True,True,True,True,True,True,False,True,True,True,False,True,False,True,True,True,False,False,True
3,True,True,False,False,False,True,False,True,True,True,True,False,True,False,False,False,True,True,True,False,True,False,False,False,True,False,False,False
4,True,True,True,False,False,True,False,True,True,True,True,True,True,True,True,False,True,True,True,False,True,False,True,True,True,False,False,True
5,True,True,True,False,False,True,False,True,True,True,True,True,True,True,True,False,True,True,True,False,True,False,True,True,True,False,False,True
6,True,True,False,False,False,True,False,True,True,True,True,False,True,False,False,False,True,True,True,False,True,False,False,False,True,False,False,False
7,True,True,False,False,True,True,False,True,True,True,True,False,True,True,True,False,True,True,True,False,True,False,False,False,True,False,False,False
8,True,True,True,False,False,True,False,True,True,True,True,False,True,True,True,False,True,True,True,False,True,False,False,True,True,False,False,False
9,True,True,True,False,False,True,False,True,True,True,True,False,True,True,True,False,True,True,True,False,True,False,False,True,True,False,False,False


In [39]:
all_features.head()

,account_id,account_name,browser,churn_date,churn_reason,collection_21_days,company_size,country,created_at,device,has_logo,industry,lead_score,max_team_size,min_team_size,mrr,os,paying,payment_currency,plan_id,region,subscription_started_at,team_size,time_diff,trial_start,user_description,user_goal,utm_cluster_id
0,1.0,"Gardner, Barron and Keller",microsoft edge,NaN,NaN,0,NaN,AU,2019-01-01 00:01:15,desktop,1,NaN,0.0,5.0,2.0,NaN,windows,0,AUD,NaN,New South Wales,NaN,NaN,11.0,2019-01-01 00:01:15,NaN,NaN,orders
1,2.0,Dunn Ltd,NaN,NaN,NaN,0,NaN,US,2019-01-01 00:01:52,mobile,1,NaN,0.0,5.0,2.0,NaN,ios,0,USD,NaN,New Jersey,NaN,NaN,NaN,2019-01-01 00:01:52,NaN,NaN,NaN
2,3.0,Boone Inc,chrome,NaN,NaN,0,NaN,US,2019-01-01 00:03:12,desktop,1,Other,0.0,1.0,1.0,NaN,windows,0,USD,NaN,Louisiana,NaN,1,-6.0,2019-01-01 00:03:12,NaN,NaN,todos
3,4.0,"Christian, Carroll and Davis",NaN,NaN,NaN,0,NaN,IL,2019-01-01 00:04:11,mobile,1,NaN,0.0,NaN,NaN,NaN,android,0,USD,NaN,Tel Aviv,NaN,NaN,NaN,2019-01-01 00:04:11,NaN,NaN,NaN
4,5.0,Brooks-Oliver,chrome,NaN,NaN,0,NaN,US,2019-01-01 00:04:21,desktop,1,Design,0.0,1.0,1.0,NaN,chrome_os,0,USD,NaN,North Carolina,NaN,1,-5.0,2019-04-04 11:09:12,NaN,NaN,todos


###Data preperation
After we created our raw features we need to make sure the fit the way our ML model expects to receive them.

In [0]:
# We map our features into different types
categorical_features = ['country', 'device']

normalized_features = ['collection_21_days']

binary_features = ['paying', 'has_logo']

untouched_features = ['account_id']

target = ['lead_score']

# And create a column transformer to handle the manipulation for us
preprocess = make_column_transformer(
    (OneHotEncoder(), categorical_features),
    (Normalizer(), normalized_features),
    (Binarizer(), binary_features)
)

###Re-splitting
We now need to split our data back to the original train set and test set.

We also make sure we keep only the columns we want in the data frame (the features)

In [45]:
# Getting only the relevant features from the dataset
dataset = all_features[categorical_features + normalized_features + binary_features + untouched_features + target]

# Filling empty values with default values 
dataset.loc[:,categorical_features] = dataset[categorical_features].fillna('')
dataset.loc[:,normalized_features +
              binary_features +
              untouched_features] = dataset[normalized_features +
                                            binary_features +
                                            untouched_features].fillna(0)

# Splitting them back up to the original train/test split
dataset_train = dataset[dataset.reset_index().account_id.isin(train_accounts.account_id)]
dataset_test = dataset[dataset.reset_index().account_id.isin(test_accounts.account_id)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [46]:
dataset.head()

,country,device,collection_21_days,paying,has_logo,account_id,lead_score
0,AU,desktop,0,0,1,1.0,0.0
1,US,mobile,0,0,1,2.0,0.0
2,US,desktop,0,0,1,3.0,0.0
3,IL,mobile,0,0,1,4.0,0.0
4,US,desktop,0,0,1,5.0,0.0


###Setting everything up
Our dataset is large (1,500,000+ accounts, each has a few users, each has events for every day)

We need to work on a smaller batch of the training data so we can iterate more quickly.

Once we find a good architecture we can increase the sample size to increase the accuracy.

In [0]:
sampled_dataset_train = dataset_train.iloc[sample_without_replacement(dataset_train.shape[0], samples_num)]

In [0]:
# We fit our column transformer on both the train and the test sets
preprocess.fit(sampled_dataset_train.append(dataset_test))

# We use transform to finally manipulate the features of our training set
X = preprocess.transform(sampled_dataset_train)

# Seperating the label
y = sampled_dataset_train.pop('lead_score')

In [0]:
# You now need to split the data into YOUR OWN training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# For standardization purposes we store y_test in a y_true variable
y_true = y_test

###Running the model
It's the money time, we can finally run our model!

First we need to created it, and train(fit) it.

In [50]:
clf = REDACTED_MODEL_TYPE(REDACTED_MODEL_PARAMETERS, random_state=42)
%time clf.fit(X_train, y_train)

NameError: ignored

In [0]:
# Now we need to get the predictions of our test set
%time y_pred = clf.predict(X_test)

###Model evaluation
Now that we have our model and it can predict the lead score based on features, we need a way to test if it's any good

####Classification report
We use classification_report to get different metrics comparing our prediction to the ground truth.

In [0]:
print(classification_report(y_true, y_pred, target_names=['Not Lead', 'Lead']))

We can also get the MCC score of the model

In [0]:
print('Acc:  {}'.format(metrics.accuracy_score(y_true, y_pred)))
print('MCC: {}'.format(metrics.matthews_corrcoef(y_true, y_pred)))
print('F1:  {}'.format(metrics.f1_score(y_true, y_pred)))

####Plotting the confusion matrix
Confusion matrices are useful for comparing our predictions

In [0]:
fig, axs = plt.subplots(ncols=2, figsize=(14,4))

cm = confusion_matrix(y_true, y_pred)
ticks = ['Not Lead', 'Lead']
cmap = sns.color_palette("Blues")

# We normalize our data to see more accurate comparsion
sns.heatmap(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], annot=True, ax=axs[0], cmap=cmap)
axs[0].set(title="Normalized confusion matrix", xlabel="Prediction", ylabel="Truth", xticklabels=ticks, yticklabels=ticks)

# We also plot the original numbers to get the whole picture
sns.heatmap(cm, annot=True, ax=axs[1], fmt='g', cmap=cmap)
axs[1].set(title="Confusion matrix", xlabel="Prediction", ylabel="Truth", xticklabels=ticks, yticklabels=ticks);

###Submitting results
After you ran several iterations, and you think your model is good enough, you can send it to us and we'll add your score on the leaderboard!

You have to get the results into the following format:
```python
{"9023749": 1, "9837598": 0, ...}
```

This is a dictionary where the keys are `account_id`s and the values are the predicted lead_score.

_Make sure you send us **all** the test accounts!_

_There should be exactly `71,683` of them!_

####Prediction
First of all, just like before, we have to predict the lead_score.

This time you need to use the test set _we_ provided.

In [0]:
submission_account_ids = dataset_test.index.values
X_submission = preprocess.transform(dataset_test).drop(columns='lead_score')

y_pred_submission = clf.predict(X_submission)

####Submission
Now that we have our submission predictions, we need to pack them up into a compatible format for our server to handle.


In [0]:
# Creating a dictionary where the keys are the account_ids
# and the values are your predictions
prediction = dict(zip(submission_account_ids, y_pred_submission))

We now just send the results to our server and wait for the score!

In [0]:
# Importing stuff for http requests
from urllib import request
import json

# We validate first that we actually send all the test accounts expected to be sent
if y_pred_submission.shape[0] != 71683 or submission_account_ids.shape[0] != 71683:
  raise Exception("You have to send all of the accounts! Expected: (71683, 71683), Got: ({}, {})".format(y_pred_submission.shape[0], submission_account_ids.shape[0]))

if "group_name" not in vars() or group_name == "":
  group_name = input("Please enter your group's name:")

data = json.dumps({'submitter': group_name, 'predictions': predictions}).encode('utf-8')

req = request.Request(f"https://leaderboard.datahack.org.il/monday/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)

res = request.urlopen(req)
print(json.load(res))